mounting drive to get data,mouting is better the uploadind data 

reading data

In [ ]:
import pandas as pd

In [ ]:
moviedf=pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv/tmdb_5000_movies.csv')

In [ ]:
moviedf.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [ ]:
creditdf=pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv/tmdb_5000_credits.csv')

In [ ]:
creditdf.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


renaming creditdf column name movie_id to id ,to merge two dataframes to single

In [ ]:
creditdf.rename({'movie_id':'id'},axis=1,inplace=True)

In [ ]:
combdf=creditdf.merge(moviedf,on='id')

In [ ]:
combdf['overview'][1]

'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.'

removing un wanted columns as we are performing content reccomendation our main column is overview column

In [ ]:
combdf.drop(columns=['homepage', 'title_x', 'title_y', 'status','production_countries'],inplace=True)

In [ ]:
combdf.shape

(4803, 18)

removing nan values

In [ ]:
combdf=combdf.dropna(axis=0)

In [ ]:
combdf.shape

(3959, 18)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf=TfidfVectorizer(min_df=3,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3),stop_words='english')

In [ ]:
tfidf_text=tfidf.fit_transform(combdf['overview'])

In [ ]:
tfidf_text

<3959x8872 sparse matrix of type '<class 'numpy.float64'>'
	with 102407 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [ ]:
sigm=sigmoid_kernel(tfidf_text,tfidf_text)

In [ ]:
len(sigm)

3959

In [ ]:
combdf['original_title'].index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4788, 4789, 4791, 4792, 4793, 4795, 4796, 4798, 4799, 4801],
           dtype='int64', length=3959)

In [ ]:
pd.DataFrame(combdf['original_title'].index,combdf['original_title'])

,0
original_title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
...,...
Bang,4795
Primer,4796
El Mariachi,4798


In [ ]:
indices =pd.Series(combdf['original_title'].index,index=combdf['original_title'])
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
Bang                                        4795
Primer                                      4796
El Mariachi                                 4798
Newlyweds                                   4799
Shanghai Calling                            4801
Length: 3959, dtype: int64

In [ ]:

indices['Spectre']

2

In [ ]:
sigm[2]

array([0.76159416, 0.76159416, 0.76164149, ..., 0.76159416, 0.76159442,
       0.76159416])

In [ ]:
len(list(enumerate(sigm[indices['Avatar']])))

In [ ]:
def give_rec(title, sig=sigm):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sigm[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:11]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return combdf['original_title'].iloc[movie_indices]

getting reccomendation of johncarter movie

In [ ]:
give_rec('John Carter')

1254                          Get Carter
2932                        Raising Cain
1307                       The Hurricane
3349                           Desperado
345                          Rush Hour 2
3068                         Rescue Dawn
91          Independence Day: Resurgence
4274    Eddie: The Sleepwalking Cannibal
581              Star Trek: Insurrection
1300                      The Ugly Truth
Name: original_title, dtype: object